In [13]:
import os
import sys
import pymysql  # MySQL connector

# Ensure the 'scripts' folder is in the Python path
sys.path.append(os.path.abspath(os.path.join("..", "scripts")))

# Import credentials securely
try:
    from config import RDS_HOST, RDS_USER, RDS_PASSWORD, RDS_DATABASE
except ModuleNotFoundError:
    raise ModuleNotFoundError("Could not find 'config.py'. Ensure it's in the 'scripts' folder.")

# Establish connection to AWS RDS MySQL
try:
    connection = pymysql.connect(
        host=RDS_HOST,
        user=RDS_USER,
        password=RDS_PASSWORD,
        database=RDS_DATABASE,
        cursorclass=pymysql.cursors.DictCursor,
        autocommit=True  # Ensures queries commit automatically
    )
    print("✅ Connected to AWS RDS successfully!")

    # Test query to verify connection
    with connection.cursor() as cursor:
        cursor.execute("SELECT DATABASE();")
        db_name = cursor.fetchone()
        print(f"Connected to database: {db_name['DATABASE()']}")

except pymysql.MySQLError as e:
    print(f"❌ MySQL Connection Error: {e}")

#finally:
 #   if 'connection' in locals() and connection.open:
  #      connection.close()
   #     print("🔌 Connection closed.")



✅ Connected to AWS RDS successfully!
Connected to database: mydb


In [15]:
import pandas as pd 

%store -r df_cleaned
# Data Loading Notebook
 

# Inserting data into the Item table
for index, row in df_cleaned.iterrows():
    item_type = row['item_type'] if pd.notna(row['item_type']) else 'Unknown'
    unit = row['unit'] if pd.notna(row['unit']) else 'Unknown'
    description = row['description'] if pd.notna(row['description']) else 'Unknown'
    location = row['location'] if pd.notna(row['location']) else 'Unknown'

    item_sql = "INSERT INTO Item (item_id, description, item_type, unit, location) VALUES (%s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE description=%s, item_type=%s, unit=%s, location=%s"
    item_values = (row['item_id'], description, item_type, unit, location, description, item_type, unit, location)

    try:
        mycursor.execute(item_sql, item_values)
    except Exception as e:
        print(f"Error inserting/updating Item: {e}. Values: {item_values}")
        mydb.rollback()
        raise

# Inserting data into the Inventory table
for index, row in df_cleaned.iterrows():
    inventory_sql = "INSERT INTO Inventory (item_id, quantity_on_hand, cost, purchase_date, vendor) VALUES (%s, %s, %s, %s, %s)"
    inventory_values = [row[col] if not pd.isna(row[col]) else None for col in ['item_id', 'quantity_on_hand', 'cost', 'purchase_date', 'vendor']]

    try:
        mycursor.execute(inventory_sql, inventory_values)
    except mysql.connector.IntegrityError as e:
        if "Duplicate entry" not in str(e):
            print(f"Integrity Error inserting into Inventory: {e}. Values: {inventory_values}")
            mydb.rollback()
            raise
    except Exception as e:
        print(f"General Error inserting into Inventory: {e}. Values: {inventory_values}")
        mydb.rollback()
        raise

# Inserting data into the Sales table
for index, row in df_cleaned.iterrows():
    sales_sql = "INSERT INTO Sales (item_id, price, date_sold, cust, quantity_sold) VALUES (%s, %s, %s, %s, %s)"
    sales_values = [row[col] if not pd.isna(row[col]) else None for col in ['item_id', 'price', 'date_sold', 'cust', 'quantity_sold']]

    try:
        mycursor.execute(sales_sql, sales_values)
    except mysql.connector.IntegrityError as e:
        if "Duplicate entry" not in str(e):
            print(f"Integrity Error inserting into Sales: {e}. Values: {sales_values}")
            mydb.rollback()
            raise
    except Exception as e:
        print(f"General Error inserting into Sales: {e}. Values: {sales_values}")
        mydb.rollback()
        raise

# Commit changes and close the connection
mydb.commit()
print("Data loaded successfully!")

# Close the connection
if mydb.is_connected():
    mycursor.close()
    mydb.close()
    print("Database connection closed.")


Error inserting/updating Item: name 'mycursor' is not defined. Values: (1000, 'Bennet Farm free-range eggs', 'D12', 'bunch', 'dozen', 'Bennet Farm free-range eggs', 'D12', 'bunch', 'dozen')


NameError: name 'mydb' is not defined

In [6]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   --- ------------------------------------ 1.6/16.1 MB 10.4 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/16.1 MB 9.7 MB/s eta 0:00:02
   -------------- ------------------------- 6.0/16.1 MB 10.5 MB/s eta 0:00:01
   --------------------- ------------------ 8.7/16.1 MB 10.8 MB/s eta 0:00:01
   ---------------------------- ----------- 11.3/16.1 MB 11.0 MB/s eta 0:00:01
   --------------------------------- ------ 13.6/16.1 MB 11.0 MB/s eta 0:00:01
   ---------------------------------------  16.0/16.1 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------  16.0/16.1 MB 11.1 MB/s eta 0:00:01
   ---------------------------------------- 16.1/16.1 MB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
print(os.getcwd())

import sys
sys.path.append(r"C:\Users\Darwin Choy\Documents\Portfolio projects\scripts")

# Set the working directory
os.chdir(r"C:\Users\Darwin Choy\Documents\Portfolio projects")

# Verify the change
print(os.getcwd())

C:\Users\Darwin Choy\Documents\Portfolio projects\notebooks
C:\Users\Darwin Choy\Documents\Portfolio projects
